In [76]:
import numpy as np
import random
from implementations import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
LOWER_BOUND = -1
UPPER_BOUND = 1

# Import data

In [64]:
DATA_FOLDER = "competition-data/"
DATA_TEST = "test.csv"
DATA_TRAIN = "train.csv"

In [80]:
from proj1_helpers import load_csv_data
y_train, x_train, ids_train = load_csv_data(DATA_FOLDER + DATA_TRAIN, LOWER_BOUND, UPPER_BOUND)

In [81]:
tx_train = x_train.T

# Cleaning data

In [9]:
'''
# constants
CATEGORICAL_FIELDS = [22]

for c in CATEGORICAL_FIELDS:
    median_rep_data_train[c] = categorical_rep_data(train_data[c])
    median_rep_data_test[c] = categorical_rep_data(test_data[c])
'''

'\n# constants\nCATEGORICAL_FIELDS = [22]\n\nfor c in CATEGORICAL_FIELDS:\n    median_rep_data_train[c] = categorical_rep_data(train_data[c])\n    median_rep_data_test[c] = categorical_rep_data(test_data[c])\n'

In [67]:
'''
column = tx_train[22]
column[column == 0] = np.nan
values, counts = np.unique(column, return_counts=True)
idx = np.argmax(counts)
most_freq = values[idx]
tx_train[tx_train == 0] = most_freq

column = tx_test[22]
column[column == 0] = np.nan
values, counts = np.unique(column, return_counts=True)
idx = np.argmax(counts)
most_freq = values[idx]
tx_test[tx_test == 0] = most_freq
'''

'\ncolumn = tx_train[22]\ncolumn[column == 0] = np.nan\nvalues, counts = np.unique(column, return_counts=True)\nidx = np.argmax(counts)\nmost_freq = values[idx]\ntx_train[tx_train == 0] = most_freq\n\ncolumn = tx_test[22]\ncolumn[column == 0] = np.nan\nvalues, counts = np.unique(column, return_counts=True)\nidx = np.argmax(counts)\nmost_freq = values[idx]\ntx_test[tx_test == 0] = most_freq\n'

In [82]:
tx_train = delete_features(tx_train, 0.9)
tx_train.shape

(30, 250000)

In [69]:
'''
import matplotlib.pyplot as plt
import numpy as np

for i in range(29):
    for j in range(i+1, 30):
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        x = tx_train[i]
        y = tx_train[j]
        plt.xlabel('Feature %d' % i)
        plt.ylabel('Feature %d' % j)
        ax.scatter(x, y)
        plt.show()
'''


"\nimport matplotlib.pyplot as plt\nimport numpy as np\n\nfor i in range(29):\n    for j in range(i+1, 30):\n        fig = plt.figure()\n        ax = fig.add_subplot(1, 1, 1)\n        x = tx_train[i]\n        y = tx_train[j]\n        plt.xlabel('Feature %d' % i)\n        plt.ylabel('Feature %d' % j)\n        ax.scatter(x, y)\n        plt.show()\n"

In [83]:
tx_train, y_train = balance(tx_train.T, y_train, LOWER_BOUND, UPPER_BOUND)
tx_train.shape

(30, 171334)

In [84]:
tx_train = replace_nan_by_median(tx_train)

In [85]:
mean_train, std_train, tx_train = standardize(tx_train)

# Cross Validation

In [78]:
from implementations import build_k_indices
#from tqdm import tqdm_notebook
from matplotlib.pyplot import figure, show
from matplotlib.ticker import MaxNLocator
import matplotlib.patches as mpatches

def cross_validation_demo():
    model = "reg_logistic_regression"
    seed = 3
    k_fold = 10
    degrees = np.arange(1, 12, 1)
    lambdas = [0]#np.logspace(-4, -2, 10)
    gammas = np.arange(0.05, 0.6, 0.05)
    #initial_w = init_w(tx_train)
    max_iters = [50, 100, 150]#np.logspace(2, 3, 4)
    batch_sizes = [64]
    k_indices = build_k_indices(y_train, k_fold, seed)
    results = []
    for degree in degrees:
        tx_train_poly = build_poly_tx(tx_train, degree)
        initial_w = init_w(tx_train_poly)
        for max_iter in max_iters:
            for batch_size in batch_sizes:
                for gamma in gammas:
                    for lambda_ in lambdas:
                        accs = []
                        ws = []
                        for k in range(k_fold):
                            w_tr, acc = cross_validation(y_train, tx_train_poly.T, initial_w,
                                                         int(max_iter),  k_indices, k, gamma, lambda_, LOWER_BOUND, UPPER_BOUND, model, batch_size)
                            ws.append(w_tr)
                            accs.append(acc)
                        w_mean = np.mean(ws, axis=0)
                        acc_mean = np.mean(accs)
                        results.append((degree, max_iter, batch_size, gamma, lambda_, acc_mean, w_mean))

                        print("Finished: " + str((degree, max_iter, batch_size, gamma, lambda_, acc_mean)))

                        '''
                        fig = plt.subplots(1, 1, figsize=(10,5))
                        plt.plot(range(1,k_fold+1), accs, marker=".", color='b', label='accuracy')
                        plt.axhline(y=acc_mean, color='r', label='mean')
                        ax = plt.gca()
                        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
                        plt.xlabel("k")
                        plt.ylabel("accuracy")
                        plt.title("k-fold accuracy for lambda=%.5f, gamma=%.2f, degree=%d" % (lambda_, gamma, degree))
                        plt.legend(loc=2)
                        plt.grid(True)
                        plt.show()
                        '''
    
    return results

results = cross_validation_demo()

/Users/valentin/Documents/EPFL/ML/Projects/CS-433/project1/implementations.py:128: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-t))


Finished: (1, 50, 64, 0.050000000000000003, 0, 0.60558571178427589)
Finished: (1, 50, 64, 0.10000000000000001, 0, 0.63918753283137808)
Finished: (1, 50, 64, 0.15000000000000002, 0, 0.6420475106519582)
Finished: (1, 50, 64, 0.20000000000000001, 0, 0.6405358080896516)
Finished: (1, 50, 64, 0.25, 0, 0.64605731629020036)
Finished: (1, 50, 64, 0.29999999999999999, 0, 0.62217358314364091)
Finished: (1, 50, 64, 0.35000000000000003, 0, 0.65976769976069571)
Finished: (1, 50, 64, 0.40000000000000002, 0, 0.62683709799801557)
Finished: (1, 50, 64, 0.45000000000000001, 0, 0.61707231658203465)
Finished: (1, 50, 64, 0.5, 0, 0.60144166228914953)
Finished: (1, 50, 64, 0.55000000000000004, 0, 0.61460923364267794)
Finished: (1, 100, 64, 0.050000000000000003, 0, 0.63904745228506388)
Finished: (1, 100, 64, 0.10000000000000001, 0, 0.63091110721998489)
Finished: (1, 100, 64, 0.15000000000000002, 0, 0.64486079495710036)
Finished: (1, 100, 64, 0.20000000000000001, 0, 0.61350026265102442)
Finished: (1, 100, 64,

KeyboardInterrupt: 

In [49]:
results.sort(key=lambda x: -x[3])
[x[3] for x in results[:4]]

[0.82701473816243332,
 0.82696770147381626,
 0.82693634368140478,
 0.8269206647851991]

In [51]:
results.sort(key=lambda x: -x[3])
best_ws = [x[4] for x in results[:4]]
mean_w = np.mean(best_ws, axis=0)

tx_test_poly = build_poly_tx(tx_test, 11)
tx_test_poly.shape

(331, 568238)

In [ ]:
### TEST SET ###
y_test, x_test, ids_test = load_csv_data(DATA_FOLDER + DATA_TEST, LOWER_BOUND, UPPER_BOUND)
tx_test = x_test.T
tx_test = delete_features(tx_test, 0.9)
tx_test = replace_nan_by_median(tx_test)
tx_test = standardize_predef(tx_test, mean_train, std_train)

# Predict labels

In [52]:
from proj1_helpers import predict_labels_kaggle

In [53]:
y_pred = predict_labels_kaggle(mean_w, tx_test_poly.T, LOWER_BOUND, UPPER_BOUND)

# Submission

In [54]:
from proj1_helpers import create_csv_submission

In [55]:
create_csv_submission(ids_test, y_pred, "test15.csv")

Test n : algorithm / features / y / w
- - - - - - - - - - - - - - - - - - - 
Test 1 : least_squares / all features standardized / y = -1,1 / random init_w

Test 2 : least_squares / corr > 0.1 features standardized / y = -1,1 / random init_w

Test 3 : least_squares_GD(10000,0.5) / all features standardized / y = -1,1 / random init_w

Test 4 : least_squares_GD(10000,0.5) / all features standardized / y = -1,1 / random init_w / poly, degree=1

Test 5 : least_squares / all features standardized / y = -1,1 / random init_w / median + categorical

Test 6 : logistic_regressoin  /all features standardized / y = 0,1 / random init_w / median + categorical + balanced

Test 7 : least_squares / all features standardized / y = 0,1 / random init_w

Test 8 : Test 1

Test 9 : Test 1

Test 10 : Test 1 / standardized test_set with mean and std from train_set

Test 11 : Test 1 / standardized test_set with mean and std from train_set / balance

Test 12 : Ridge regression / non-balanced / standardized test_set with mean and std from train_set / y=-1,1 / mean of 4 best lambdas for degree 11

Test 13 : Ridge regression / balanced / standardized test_set with mean and std from train_set / y=-1,1 / mean of 4 best lambdas for degree 11

Test 14 : Ridge regression / balanced before doing anything / standardized test_set with mean and std from train_set / y=-1,1 / mean of 4 best lambdas for degree 11

Test 15 : Ridge regression / Removed all rows containing at least a NaN / balanced before doing anything / standardized test_set with mean and std from train_set / y=-1,1 / mean of 4 best lambdas for degree 11 / Replaced NaN values in test_set by median in test_set

## Further work

- balance output (batch numpy)
- median and category
- features engineering : features d'intéraction
- logistic regression 